In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Data Done Right: 🚀 From Zero to Insights, Fast.

## Introduction

Learn how to build a scalable data stack that delivers actionable insights and powers AI-driven innovation. See it in action with a example use case that transforms raw data into business impact.

This notebook demonstrates a simple but powerful use case example on Google Cloud. You'll learn how to:

* **Get Data from an Environmental API**
* **Create Synthetic Data with Gemini**
* **Data Ingestion & Analysis with BigQuery**
* **How to train a Machine Learning model with SQL**
* **How to use LLMs in your Data Warehouse**

##Scenario

### Building a Happiness Prediction App

Imagine you're relocating.  One of the biggest questions is: how happy will I be?  This notebook tackles this challenge by building a happiness prediction app. Our core hypothesis is that air quality significantly impacts well-being, and therefore, happiness. We use real-world data on location, air quality, and happiness, supplementing it with synthetic data generated by a Large Language Model.  We then leverage the power of SQL within our data warehouse to train a machine learning model.  Finally, we demonstrate how to use this model to predict happiness scores for specific locations.

*Diclaimer: the use case in this notebook was designed as inspiration. Some steps including the machine learning training on the given feature set might not represent a realistic scenario in the 'real world'.*

## What you'll need

* A Google Cloud Account and Google Cloud Project

## Basic Setup
### Install dependencies

In [ ]:
%pip install \
  google-cloud-aiplatform \
    --quiet

### Connect Your Google Cloud Project

In [ ]:
# @markdown Please fill in the value below with your GCP project ID and then run the cell.

# Please fill in these values.
project_id = ""  # @param {type:"string"}

# Quick input validations.
assert project_id, "⚠️ Please provide a Google Cloud project ID"

# Configure gcloud.
!gcloud config set project {project_id}

### Authenticate to Google Cloud within Colab
If you're running this on google colab notebook, you will need to Authenticate as an IAM user.

In [ ]:
import google.auth

credentials, project_id = google.auth.default()

## Get Air Quality data for Google Form responses.

In a previous step, we used Google Form to gather data from people about their home location and their happiness. The CSV file *FormResponses.csv* represents the CSV export from the Google Sheets that is connected to our Google Form.
(Documentation on how to use Google Forms with Google Sheets [here](https://support.google.com/docs/answer/2917686?sjid=18278159077155361057-EU))

See how you can create your Environmental API key [here](https://developers.google.com/maps/documentation/javascript/get-api-key).

In [ ]:
# @markdown Please fill in the value below with your environmental API key and then run the cell.

# Please fill in these values.
environmental_api_key = ""  # @param {type:"string"}

# Quick input validations.
assert project_id, "⚠️ Please provide a Google Cloud Environmental API Key"


In [ ]:
import pandas as pd
import requests
import json

# Load the happiness survey data
df = pd.read_csv('/content/FormResponses.csv') #The name of your CSV file

# Extract location data & Happy Score form the CSV file
latitudes = df['Your Address: Location Latitude (please use this format: 37.419734)'].values
longitudes = df['Your Address: Location Longitude (please use this format: -122.0827784)'].values
happy_score = df['How happy do you feel when spending time at home and in your neighborhood?'].values

# AirQuality API endpoint and your API key
api_endpoint = "https://airquality.googleapis.com/v1/currentConditions:lookup"
api_key = environmental_api_key

In [ ]:
def get_air_quality(latitude, longitude):
  """
  Fetches air quality data for a given latitude and longitude using the AirQuality API.
  Includes extended data (pollutants).
  """

  params = {
      "key": api_key
  }
  data = {
      "location": {
          "latitude": latitude,
          "longitude": longitude
      },
      "extraComputations": [
        "HEALTH_RECOMMENDATIONS",
        "DOMINANT_POLLUTANT_CONCENTRATION",
        "POLLUTANT_CONCENTRATION",
        "LOCAL_AQI",
        "POLLUTANT_ADDITIONAL_INFO"
      ],
      "languageCode": "en"
  }
  response = requests.post(api_endpoint, params=params, json=data)
  response.raise_for_status()  # Raise an exception for bad status codes
  data = response.json()
  print (data)

  # Access the pollutants data
  pollutants = data['pollutants']

  # Iterate through the pollutants and extract the values
  for pollutant in pollutants:
      code = pollutant['code']
      display_name = pollutant['displayName']
      full_name = pollutant['fullName']
      value = pollutant['concentration']['value']
      units = pollutant['concentration']['units']

      #print(f"{display_name} ({full_name}):")
      #print(f"  Value: {value} {units}")

      # Instead of printing, add the data to a dictionary
      pollutant_data = {
          "code": code,
          "display_name": display_name,
          "full_name": full_name,
          "value": value,
          "units": units
      }
      data['pollutants'].append(pollutant_data)

      return data

In [ ]:
# Fetch air quality data for each location from the CSV file.
air_quality_data = []
for lat, lon, h_score in zip(latitudes, longitudes, happy_score):
    data = get_air_quality(lat, lon)
    print(data)
    if data:
        pollutants_data = {}
        for pollutant in data['pollutants']:
            # Check if 'concentration' key exists before accessing it
            if 'concentration' in pollutant:
                pollutants_data[pollutant['code']] = pollutant['concentration']['value']
        # Extract individual pollutants (if available) or use None if not found
        pm25 = pollutants_data.get('pm25')
        no2 = pollutants_data.get('no2')
        o3 = pollutants_data.get('o3')
        # ... extract other pollutants as needed ...
        air_quality_data.append([lat, lon, pm25, no2, o3, h_score])  # Append data to the list
    else:
        # Handle the case where no data was returned for the location
        air_quality_data.append([lat, lon, None, None, None, h_score])  # Append None values

print(air_quality_data)

In [ ]:
import csv

# Write the data to a new CSV file (without latitude and longitude)
with open('air_quality_data.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['PM2.5', 'NO2', 'O3', 'happy_score'])  # Write header row (without lat/lon)
    for row in air_quality_data:
        writer.writerow(row[2:])  # Write data rows (excluding lat/lon)

In [ ]:
with open('air_quality_data.csv', 'r') as file:
    reader = csv.reader(file)
    for i in range(5):  # Read the first 5 rows
        print(next(reader))

## Create Synthetic Data with Gemini

As we only were able to collect a few data points with our Google Form, we decide to create synthetic data that is based on our form responses. Doing so, we can ensure to have enough data points to train our machine learning model.

In [ ]:
#Defining the JSON response schema that we expect from the Gemini output.
response_schema = {
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "PM2.5": {
                "type": "number",
            },
            "NO2": {
                "type": "number",
            },
            "O3": {
                "type": "number",
            },
            "Happy_Score": {
                "type": "integer",
            },
        },
        "required": [  # List of required properties
            "PM2.5",
            "NO2",
            "O3",
            "Happy_Score"
        ],
    },
}

More information about the model parameters of the foundation models can be found [here](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/content-generation-parameters).

In [ ]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part, GenerationConfig

# Set up the model and generation configuration
generation_config = vertexai.generative_models.GenerationConfig(
    max_output_tokens=8192,
    temperature=0.5,
    top_p=0.95,
    response_mime_type="application/json",
    response_schema=response_schema
)

In [ ]:
import csv

# Read the CSV file
with open('air_quality_data.csv', 'r', newline='') as csvfile:
    reader = csv.reader(csvfile)
    data = list(reader)  # Read all rows into a list

# Write the data to a TXT file to being able to send it to the Gemini API (CSV files were not supported when this notebook was created)
with open('air_quality_data.txt', 'w') as txtfile:
    for row in data:
        txtfile.write('\t'.join(row) + '\n')  # Join values with tabs and add newline

### Calling the Gemini API to generate synthetic data

For more information and best practices regarinding prompting, check out the documentation [here](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/introduction-prompt-design).

In [ ]:
import base64
import json

global json_string

def generate():
    vertexai.init(project=f"{project_id}", location="europe-west3")
    model = GenerativeModel(
        "gemini-1.5-flash-002",
        system_instruction=["""You are a assistant that helps me create a synthetic dataset and output it in JSON format."""]
    )
    responses = model.generate_content(
        ["""Task: Generate Synthetic Dataset for Linear Regression Source Data:""", document1, text1],
        generation_config=generation_config,
        stream=True,
    )

    json_string = ""
    for response in responses:
       print(response.text, end="")
       json_string += response.text

    # Attempt to parse the accumulated string as JSON
    try:
        json_object = json.loads(json_string)
        return json.dumps(json_object)  # Return the re-serialized JSON string
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        # Handle the error appropriately, e.g., return an error message or None
        return None

# Read the TXT file
txt_file_path = "/content/air_quality_data.txt"

with open(txt_file_path, "r") as f:
    txt_data = f.read()

encoded_data = base64.b64encode(txt_data.encode("utf-8")).decode("utf-8")
document1 = Part.from_data(
    mime_type="text/plain",
    data=encoded_data
)

text1 = """Dataset Requirements:
1. Context:
  -The attached txt. file includes the initial data points for the dataset.
  -The dataset consists of air quality data for a specific location and the corresponding happy score from one person.
2. Data Enrichment:
  - Randomly generate additional, diverse data points to expand the dataset.
  - Ensure these new data points align with the distribution and characteristics of the original data points.
3. Feature Selection:
  - Include only these original features in the final synthetic dataset:
    - PM2.5
    - NO2
    - O3
    - Happy_Score (from 1 as very low and 5 as very high)
4. Data Types and Relationships:
  - Ensure the relationships between the Air Quality features and the happy score are suitable for linear regression.
5. Output:
  - Generate a JSON file containing 50 data points."""

In [ ]:
# Call the functions
json_string = generate()

In [ ]:
import csv
import json

def write_to_csv(json_string):
    """Converts a JSON string to a CSV file.

    Args:
      json_string: A JSON string containing the data.
    """

    try:
        # 1. Parse the outer JSON string
        data = json.loads(json_string)

        # 2. Extract column names from the first item in the list
        column_names = list(data[0].keys())

        # 3. Write the data to a CSV file
        with open('synthetic_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
            csv_writer = csv.writer(csvfile)
            csv_writer.writerow(column_names)  # Write header row

            # Write data rows (iterate through the list of dictionaries)
            for row in data:
                csv_writer.writerow(row.values())

    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
    except KeyError as e:
        print(f"Error: Key not found - {e}")

In [ ]:
write_to_csv(json_string)

### Previewing our newly generated CSV file consisting of the Air Quality metrics and the happiness score.

In [ ]:
import csv

with open('synthetic_data.csv', 'r') as file:
    reader = csv.reader(file)
    for i in range(5):  # Read the first 5 rows
        print(next(reader))

## Set up BigQuery as your Data Warehouse
In the next step we will import our dataset to a newly created dataset in BigQuery.
Please set the following variables.

In [ ]:
# @markdown Please fill in the both the Google Cloud region and name of your BigQuery dataset and table.

# Please fill in these values.
multi_region = "eu"  # @param {type:"string"}
dataset_name = "happy_air"  # @param {type:"string"}
table_name = "synthetic_data"  # @param {type:"string"}

In [ ]:
# Quick input validations.
assert multi_region, "⚠️ Please provide a Google Cloud multi-region"
assert dataset_name, "⚠️ Please provide the name of your dataset"
assert table_name, "⚠️ Please provide the name of your table"

In [ ]:
# Create a BigQuery Client for your project

from google.cloud import bigquery

client = bigquery.Client(project=project_id)

### Create the BigQuery Dataset
If you have already created an BigQuery Dataset, you can skip these steps and skip to the `Ingest data to your BigQuery dataset` section.

> ✨ We do use the command line bq command.

In [ ]:
!bq mk --location=$multi_region --dataset $project_id:$dataset_name

Let's check if the dataset is existing. Just list all the datasets in the project.

In [ ]:
for dataset in client.list_datasets():
  print(dataset.dataset_id)

## Ingest your CSV file into BigQuery

This function will ingest your csv file into your BigQuery dataset.
For documentation on how to load data into BigQuery check out [this link](https://https://cloud.google.com/bigquery/docs/loading-data).

In [ ]:
# TODO(developer): Set table_id to the ID of the table to create.
table_id = f"{project_id}.{dataset_name}.{table_name}"

# TODO(developer): Set the file path to your local CSV file
file_path = "/content/synthetic_data.csv"

job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    autodetect=True
)

with open(file_path, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_id, job_config=job_config)

job.result()
  # Waits for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id)
)

## Analyse your data with Data Canvas

For this exercise, head over to BigQuery Studio in your Google Cloud Console ([Link to BigQuery Studio](https://console.cloud.google.com/bigquery)).



1.   In the *BigQuery Explorer* navigate to your dataset.
2.   Click on the toggle button for your dataset to see your table.
3.   Click on the three dots for your table and select 'Query in Data Canvas'
4.   A new tab with *Data Canvas* will open.
5.   Begin to *'Query'*  your data with natural language prompts.


Example prompts:

*  **Show me the top 10 happiest data points**
*  **What is the average happiness score?**
*  **What is the maximum PM2.5 value?**
*  **Show me the rows where the happiness score is higher and the PM2.5 level is also higher in relation to the average values.**






## Train your Machine Learning model with SQL code
####In the next steps we will train a linear regression model with the synthetic dataset we just ingested into our BigQuery table.



In [ ]:
# @markdown Please fill in the name of your linear regression model.

# Please fill in these values.
model_name = "ml_model_happy"  # @param {type:"string"}

In [ ]:
ml_model_id = f"{project_id}.{dataset_name}.{model_name}"

#### The SQL statements will be executed via Python Magics for BigQuery ([Link to documentation](https://cloud.google.com/python/docs/reference/bigquery/latest/magics) )

The following SQL query uses BigQuery ML to train a linear regression model ([Docs for BigQuery ML](https://cloud.google.com/bigquery/docs/bqml-introduction)). We select all the features from our initial dataset and include 'Happy_Score' at the end as the value to predict.

**Disclaimer:** *Our example features for training the Happy Air model are not quite realistic as all the features (the metrics regarding the air quality) are highly correlated. In an ideal example you would have a more diverse feature set to train your model. This would allow the model to learn more complex relationships and potentially make more accurate predictions.*

In [ ]:
query = f"""
CREATE OR REPLACE MODEL `{ml_model_id}`
OPTIONS
  (model_type='linear_reg',
  input_label_cols=['Happy_Score']) AS
SELECT
  PM2_5,
  NO2,
  O3,
  Happy_Score
FROM
  `{table_id}`
"""
print (query)
query_job = client.query(query)
results = query_job.result()

When navigating to your dataset in the Google Cloud console you will find your freshly trained model. Clicking on it you will find information about training details and evaluation metrics. You can also use the ML.Evaluate function to get evaluation metrics ([Documentation here](https://cloud.google.com/bigquery/docs/evaluate-overview))

In [ ]:
print(ml_model_id)

In [ ]:
%%bigquery --project $project_id
SELECT * FROM ML.EVALUATE(MODEL `{hard-code your ml_model_id here}`);

After having our prompt column in place we need to create a model within BigQuery to be able to utilize GenAI capabilities.

Firstly, we need to create an external connection within BigQuery to connect remotely to Vertex AI models.

##Create a connection to Vertex AI and create a LLM model in your BigQuery dataset.

In [ ]:
# @markdown Please fill in the name of your connection to Vertex AI.

# Please fill in these values.
connection_name = "connection_vertex"  # @param {type:"string"}

In [ ]:
!bq mk --connection --location=$multi_region --project_id=$project_id \
    --connection_type=CLOUD_RESOURCE $connection_name

After creating the connection to Vertex AI (and before importing the Gemini model into your BigQuery dataset) we have to grant the service account associated with the new connnection the relevant permissions to use Vertex AI:


1.   Go to the [BigQuery console](https://console.cloud.google.com/bigquery).
2.   In the *Explorer* tab on the left of your screen navigate to your *Project_ID* and click on it to show all the associated resources.
3.   Click on *External Connections* and select the connection we just created.
4.   The *Connection Info* will be displayed. Copy the *Service account id*.
5.   Navigate to [IAM in the Google Cloud Console](https://console.cloud.google.com/iam-admin/iam).
6.   Click on *+ Grant Access* and paste the *Service account id* from the previous steps into the *New Principal* field.
7.   Select *Vertex AI User* as the role and click on *Save*.
8.   Restart the Colab or Workbench Runtime and reload the page.
9.   Execute the *Basic Setup* section of this notebook to authenticate again.






**Import Gemini 2.0 Flash into your BigQuery dataset using the created connection to Vertex AI.**

In [ ]:
# @markdown Please fill in the name of your connection to Vertex AI.

# Please fill in these values.
gemini_model_bq = "gemini-2.0-flash-001"  # @param {type:"string"}


In [ ]:
connection_id=f"projects/{project_id}/locations/{multi_region}/connections/{connection_name}"

In [ ]:
query = f"""
CREATE OR REPLACE MODEL `{project_id}.{dataset_name}.gemini_model_bq`
  REMOTE WITH CONNECTION `{connection_id}`
  OPTIONS (ENDPOINT = '{gemini_model_bq}');
"""

try:
  query_job = client.query(query)
  query_job.result()  # Wait for the query to complete

  # Check for errors in the job configuration or execution
  if query_job.errors:
    for error in query_job.errors:
      print(f"Error creating remote model: {error}")
  else:
    print("Remote model created successfully!")

except Exception as e:
  print(f"An unexpected error occurred: {e}")

## Get predictions from your trained model and transform the output with Gemini.


**Paste in your latitude and longitude values so we can feed it into our linear regression model.**

In [ ]:
# @markdown ### **Location Data:**

Latitude = 52.51534871316592 # @param {type:"number"}
Longitude = 13.396667668241038 # @param {type:"number"}


# Combine features into a dictionary
feature_names = [
    "PM2_5",
    "NO2",
    "O3"
]

global feature_values

# Fetch air quality data for the location
new_data = get_air_quality(Latitude, Longitude)
print(new_data)

new_air_quality_data = []

if new_data:
    new_pollutants_data = {}
    for pollutant in new_data['pollutants']:
        # Check if 'concentration' key exists before accessing it
        if 'concentration' in pollutant:
            new_pollutants_data[pollutant['code']] = pollutant['concentration']['value']
    # Extract individual pollutants (if available) or use None if not found
    pm25 = new_pollutants_data.get('pm25')
    no2 = new_pollutants_data.get('no2')
    o3 = new_pollutants_data.get('o3')

    feature_values = [pm25, no2, o3]

features = {name: value for name, value in zip(feature_names, feature_values)}
print(features)
# Now you can use the 'features' dictionary in your BigQuery query

In this step we use the BigQuery client again to store the output in a dataframe.
The query consists of two parts:


1.   The inner part of the query gets a prediction from the linear regression model. As input we're using the features you selected in the cell above.
2.   The outer part of the query uses the predicted value to get an interpretation of the result from the Gemini API.



*Please make sure to insert your model ids into the SQL query.*

In [ ]:
query = f"""
SELECT * FROM ML.GENERATE_TEXT(
MODEL `your_project_id.your_dataset.your_gemini_model`,
(
    SELECT CONCAT("You are an AI assistant helping users understand how happy a person gets at a location based on AirQuality Data. Given the following information:The happiness level from 1 (being low) to 5 (being high) is: ", predicted_Happy_score, ". The AirQuality variable at the location are: ",
    "{', '.join(f'{name}= {value}' for name, value in features.items())}. ",
    "Generate a response that explains the predicted Happy_Score in a clear and friendly manner. Include insights into how the AirQuality might have influenced the score. Offer a funny suggestion on top. Answer in 6 sentences. Output the key words in bold and use paragraphs.")
    AS prompt
    FROM (
    SELECT *
    FROM ML.EXPLAIN_PREDICT(
        MODEL `your_project_id.your_dataset.your_linearregression_model`,
        (SELECT {', '.join(f'{value} AS {name.replace(" ", "_")}' for name, value in features.items())}),
        STRUCT(3 as top_k_features)
    ))),
    STRUCT(
      0.5 AS temperature,  -- Controls randomness (0.0 - 2.0)
      8192 AS max_output_tokens, -- Limits the generated text length
      0.95 AS top_p  -- Controls diversity of generated text
    )
);
"""
print (query)
df = client.query(query).to_dataframe()

## Your results

In [ ]:
import json
from IPython.display import Markdown

generated_output = df ["ml_generate_text_result"][0]

# Parse the JSON string
data = json.loads(generated_output)

# Extract the text
text = data['candidates'][0]['content']['parts'][0]['text']

display(Markdown(text))